In [51]:
# read image data
import cv2
def read_image(path):
    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    return img

In [52]:
import json
def get_die_data(path):
    data = ""
    with open(path) as f:
        data = json.load(f)
    die_data,care_areas,exclusion_zones = data['die'],data['care_areas'],data['exclusion_zones']
    return die_data,care_areas,exclusion_zones

In [53]:
die_data,care_areas,exclusion_zones = get_die_data("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/input.json")
# print(img)
print(die_data)
print(care_areas)
print(exclusion_zones)

{'width': 800, 'height': 600, 'rows': 1, 'columns': 5}
[{'top_left': {'x': 0, 'y': 600}, 'bottom_right': {'x': 800, 'y': 0}}]
[]


In [54]:
img = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/wafer_image_1.png")
print(img.shape)

(600, 800)


In [55]:
# combine all image
import numpy as np
def combine_image():
    c = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/wafer_image_1.png")
    for i in range(2,6):
        img = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_1_data/wafer_image_"+str(i)+".png")
        c = np.concatenate((c,img),axis=1)
#     c = np.transpose(c)
    return c

In [56]:
# get all individual dies
import numpy as np
def individual_dies():
    c = combine_image()
    (j,i) = c.shape
    dies = dict()
    dies_count = 1
    for ind in range(0,i,die_data['width']):
        a = c[:,ind:ind+die_data['width']]
        dies[str(dies_count)] = a
        dies_count+=1
    return dies

In [57]:
# return optimal die

#calculate the frequency of pixel value at each coordinates
def calculate_freq():
    freq = []
    for i in range(die_data['height']):
        a = []
        for j in range(die_data['width']):
            s = dict()
            a.append(s)
        freq.append(a)

    all_dies = individual_dies()
    for [key,value] in all_dies.items():
        for i in range(die_data['height']):
            for j in range(die_data['width']):
                if str(value[i][j]) in freq[i][j]:
                    freq[i][j][str(value[i][j])]+=1
                else:
                    freq[i][j][str(value[i][j])]=1



    return freq

In [58]:
# return optimal die
def get_optimal_die():
    freq = calculate_freq()
    optimal_die = np.zeros((die_data['height'],die_data['width']))
    for i in range(die_data['height']):
        for j in range(die_data['width']):
            max_key = max(freq[i][j],key=freq[i][j].get)
            optimal_die[i][j] = max_key
            
    return optimal_die

In [59]:
# calculate error
# given all dies and optimal dies, calculate the error
def calculate_errors():
    optimal_die = get_optimal_die()
    all_dies = individual_dies()
    errors = []
    for [key,value] in all_dies.items():
        for i in range(die_data['height']):
            for j in range(die_data['width']):
                if value[i][j] != optimal_die[i][j]:
                    error = []
                    error.append(key)
                    error.append(j+1)
                    error.append(i+1)
                    errors.append(error)
    return errors

In [60]:
# write file to csv
import csv
def make_csv_file():
    errors = calculate_errors()
    with open('answer.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(errors)

In [61]:
make_csv_file()